## Exercise 3

### Installation of ffmpeg on windows machine

https://www.ffmpeg.org/download.html#build-windows
following link to 
https://www.gyan.dev/ffmpeg/builds/

Four options available:

git full - built from master branch with a large set of libraries
git essentials - built from master branch with commonly-used libraries
release full - built from latest release branch with a large set of libraries
release essentials - built from latest release branch with commonly-used libraries downloaded the following package

Opted for the full one
https://www.gyan.dev/ffmpeg/builds/packages/ffmpeg-2021-08-18-git-dbe40478e2-full_build.7z

This also contains ffprobe

In [1]:
from os import walk
import pathlib


import sys

!{sys.executable} -m pip install ffprobe-python
!{sys.executable} -m pip install ffmpeg-python


In [2]:
from ffprobe import FFProbe
import ffmpeg

In [20]:
class Report:
    def __init__(self, filePath):
        self.filePath = filePath
        self.fixedFilePath = ''
        self.isOkAudioStream = True
        self.isOkVideoStream = True
        self.isOkContainer = True
        self.isOkVideoCodec = True
        self.isOkAudioCodec = True
        self.isOkFrameRate = True
        self.isOkAspectRatio = True
        self.isOkResolution = True
        self.isOkVideoBitRate = True
        self.isOkAudioBitRate = True
        self.report = ''
        
    def isOk(self):
        return (self.isOkAudioStream and
            self.isOkVideoStream and
            self.isOkContainer and
            self.isOkVideoCodec and
            self.isOkAudioCodec and
            self.isOkFrameRate and
            self.isOkAspectRatio and
            self.isOkResolution and
            self.isOkVideoBitRate and
            self.isOkAudioBitRate)
        

def CheckFile(filePath):
    report = Report(filePath)
    strReport = f'Report for file {filePath}\n'
    
    audioStream = None
    videoStream = None
    
    extension = pathlib.Path(report.filePath).suffix
    if extension != '.mp4':
        report.isOkContainer = False
        strReport += f'-- Wrong video container format. Expected mp4, found {extension}\n'
    
    metadata = FFProbe(filePath)
    
    for s in metadata.streams:
        if s.is_audio():
            audioStream = s
        elif s.is_video():
            videoStream = s
            
    if audioStream == None:
        report.isOkAudioStream = False
        strReport += '-- No audio stream in file\n'
    else:
        if audioStream.codec() != 'aac':
            report.isOkAudioCodec = False
            strReport += f'-- Wrong codec. Expected aac, found {audioStream.codec()}\n'
            
        audioBitRate = int(audioStream.bit_rate) / 1000
        if (audioBitRate != 256):
            report.isOkAudioBitRate = False
            strReport += f'-- Wrong audio bit rate. Expected 256 kbps, found {audioBitRate} kbps\n'
        
    if videoStream == None:
        report.isOkVideoStream = False
        strReport += '-- No video stream in file\n'
    else:
        if videoStream.codec() != 'h264':
            report.isOkVideoCodec = False
            strReport += f'-- Wrong codec. Expected h.264, found {videoStream.codec()}\n'
        
        frameRate = round(videoStream.frames() / videoStream.duration_seconds())
        if frameRate != 25:
            report.isOkFrameRate = False
            strReport += f'-- Wrong frame rate. Expected 25, found {frameRate}\n'
        
        aspectRatio = videoStream.frame_size()[0] / videoStream.frame_size()[1]
        if aspectRatio != 16/9:
            report.isOkAspectRatio = False
            strReport += f'-- Wrong aspect ratio. Expected 16/9 ({16/9}), found {aspectRatio}\n'
        
        if videoStream.frame_size() != (640, 360):
            report.isOkResolution = False
            strReport += f'-- Wrong resolution. Expected 640x360, found {videoStream.frame_size()[0]}x{videoStream.frame_size()[1]}\n'
        
        videoBitRate = int(videoStream.bit_rate) / 1000000
        if (videoBitRate < 10 or videoBitRate > 15):
            report.isOkVideoBitRate = False
            strReport += f'-- Wrong video bit rate. Expected between 10 and 15 Mbps, found {videoBitRate} Mbps\n'
    strReport += '---------------------------------------------\n\n'
    report.report = strReport
    return report






def FixFile(report):
    if report.isOk():
        return
    
    extension = '.mp4'
    report.fixedFilePath = f'{report.filePath[0: -len(extension)]}_formatOK{extension}'

    #if report.isOkVideoBitRate == False:
    #    arguments += 'video_bitrate=12000 '
        
    #if report.isOkAudioBitRate == False:
    #    arguments += 'audio_bitrate=256 '
        
        
    video = ffmpeg.input(report.filePath)
    
    #if report.isOkFrameRate == False:
    video = video.filter('fps', fps=25, round='up')
        
        
    #if report.isOkAspectRatio == False or report.isOkResolution == False:
    video = video.filter('scale', width='640', height='360')
          
    video.output(f'{report.filePath[0: -len(extension)]}_formatOK{extension}', vcodec="libx264", acodec="aac", video_bitrate=12000000, audio_bitrate=256).overwrite_output().run()


    #print(out)
    

In [21]:
    
filmFolderPath = "Exercise3_Films"

filenames = next(walk(filmFolderPath), (None, None, []))[2]

for f in filenames:
    
    if 'OK' in f:
        continue
        
    filePath = f'{filmFolderPath}\\{f}'
    report = CheckFile(filePath)
    
    print(report.report)
    
    FixFile(report)
    
    report = CheckFile(report.fixedFilePath)

    print(report.report)
    

Report for file Exercise3_Films\Cosmos_War_of_the_Planets.mp4
-- Wrong audio bit rate. Expected 256 kbps, found 317.103 kbps
-- Wrong frame rate. Expected 25, found 30
-- Wrong aspect ratio. Expected 16/9 (1.7777777777777777), found 1.7740112994350283
-- Wrong resolution. Expected 640x360, found 628x354
-- Wrong video bit rate. Expected between 10 and 15 Mbps, found 2.989377 Mbps
---------------------------------------------


Report for file Exercise3_Films\Cosmos_War_of_the_Planets_formatOK.mp4
-- No audio stream in file
---------------------------------------------


Report for file Exercise3_Films\Last_man_on_earth_1964.mov
-- Wrong video container format. Expected mp4, found .mov
-- Wrong codec. Expected aac, found pcm_s16le
-- Wrong audio bit rate. Expected 256 kbps, found 1536.0 kbps
-- Wrong codec. Expected h.264, found prores
-- Wrong frame rate. Expected 25, found 24
-- Wrong video bit rate. Expected between 10 and 15 Mbps, found 9.285191 Mbps
--------------------------------